In [1]:
import pandas as pd 
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn import preprocessing
import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
dataset_2014=pd.read_csv("Demographics and 311 calls merged by NTA - Normalized.csv")

In [3]:
dataset_2014.head()

,Neighborhood,Normalized 311 calls,Median House Value,Median Age,White only,Black only,Median Rent,Median Income,Normalized cars,Normalized High school,Normalized bachelors,Normalized masters,Normalized PHD,Normalized familiy household,Normalized non-family household,Normalized owner ocuppied,Normalized renter ocuppied,Total Population,Total households
0,Airport,inf,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,Allerton-Pelham Gardens,0.139654,445200.0,39.70,11779.0,11280.0,1538.0,64702.0,1.129863,170.556606,103.757977,52.558497,7.356417,0.770105,0.229895,0.601616,0.398384,33848.0,9587.0
2,Annadale-Huguenot-Prince's Bay-Eltingville,0.169064,527900.0,41.60,27441.0,152.0,1342.0,91354.5,1.858472,210.446054,145.184210,57.879597,5.164108,0.798786,0.201214,0.840965,0.159035,28853.0,9885.0
3,Arden Heights,0.147098,409750.0,40.45,23119.0,410.0,1299.0,88864.5,1.741741,188.149363,152.136552,69.603937,5.507382,0.797656,0.202344,0.908014,0.091986,25602.0,8960.0
4,Astoria,0.184188,637150.0,34.10,53369.0,4421.0,1432.0,57798.0,0.422238,141.163500,217.835138,89.772359,10.999416,0.467531,0.532469,0.170327,0.829673,77095.0,34310.0


In [4]:
len(dataset_2014)

195

In [5]:
dataset_2014_=pd.read_csv("Demographics and 311 calls merged by NTA - Normalized.csv").dropna()
len(dataset_2014_)

190

100 rows lost when dropping nan values

In [6]:
dataset_2014_.columns

Index([u'Neighborhood', u'Normalized 311 calls', u'Median House Value',
       u'Median Age', u'White only', u'Black only', u'Median Rent',
       u'Median Income', u'Normalized cars', u'Normalized High school',
       u'Normalized bachelors', u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households'],
      dtype='object')

In [7]:
X=dataset_2014_[dataset_2014_.columns[3:23]]
Y=dataset_2014_[dataset_2014_.columns[2]]

In [8]:
X_scaled = preprocessing.scale(X)
Y_scaled = preprocessing.scale(Y)

In [9]:
#LASSO
X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.30, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
R2_store=[]
for i in range(-300,10):
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
    R2_store.append(R2_OS)
optim_alpha=range(-300,10)[np.where(R2_store==np.max(R2_store))[0][0]]
print "alpha = %f"%optim_alpha
Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
Lasso.fit(X_train,Y_train)
R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
print R2_OS

alpha = 0.000000
0.611655683366


C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Users\juanpablo\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:454: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  positive)
C:\Users\juanpablo\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


In [10]:
Lasso.coef_

array([ -1.81980306e-01,   3.14065432e-01,   1.38926812e-01,
         2.03209883e-01,   2.05637107e-01,  -3.35051869e-02,
         1.10388181e-01,   3.53241679e-01,   1.97065222e-01,
         2.22203621e-01,   4.30948810e-01,  -1.17978941e-14,
        -3.03754862e-01,  -3.11044485e-11,  -6.62385295e-02,
        -1.12865059e-02])

In [11]:
coefficients_result=zip(Lasso.coef_,X.columns)

In [12]:
coefficients_table=pd.DataFrame()
coefficients_table['results']=Lasso.coef_
coefficients_table['labels']=X.columns
coefficients_table.sort('results',inplace=True)
C_table=coefficients_table[abs(coefficients_table.results)>0]
C_table

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,results,labels
12,-3.037549e-01,Normalized owner ocuppied
0,-1.819803e-01,Median Age
14,-6.623853e-02,Total Population
5,-3.350519e-02,Normalized cars
15,-1.128651e-02,Total households
13,-3.110445e-11,Normalized renter ocuppied
11,-1.179789e-14,Normalized non-family household
6,1.103882e-01,Normalized High school
2,1.389268e-01,Black only
8,1.970652e-01,Normalized masters


#Original 311-2014 file

In [13]:
complete_2014=pd.read_csv("311 - 2014 complaints with CT.csv")

C:\Users\juanpablo\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (18,41,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
complete_2014.head()

,Unnamed: 0,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,...,BoroCode,BoroName,CDEligibil,CT2010,CTLabel,NTACode,NTAName,PUMA,Shape_Area,Shape_Leng
0,0,29611713,12/31/2014 12:00:00 AM,06/05/2015 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,UNSANITARY CONDITION,MOLD,RESIDENTIAL BUILDING,10019.0,...,1.0,Manhattan,I,13700.0,137.0,MN17,Midtown-Midtown South,3807.0,2.435168e+06,6257.852032
1,1,29607489,12/31/2014 12:00:00 AM,01/22/2015 08:56:16 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,1-2 Family Dwelling,11205.0,...,3.0,Brooklyn,E,23300.0,233.0,BK75,Bedford,4003.0,1.849442e+06,5438.790291
2,2,29609686,12/31/2014 12:00:00 AM,12/31/2014 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10040.0,...,1.0,Manhattan,E,27700.0,277.0,MN35,Washington Heights North,3801.0,1.760156e+06,5903.151960
3,3,29608954,12/31/2014 12:00:00 AM,01/03/2015 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11373.0,...,4.0,Queens,E,46700.0,467.0,QN29,Elmhurst,4107.0,1.528978e+06,6450.862938
4,4,29609736,12/31/2014 12:00:00 AM,06/05/2015 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT/PLASTER,WALL,RESIDENTIAL BUILDING,10019.0,...,1.0,Manhattan,I,13700.0,137.0,MN17,Midtown-Midtown South,3807.0,2.435168e+06,6257.852032


In [15]:
complete_2014.columns

Index([u'Unnamed: 0', u'Unique Key', u'Created Date', u'Closed Date',
       u'Agency', u'Agency Name', u'Complaint Type', u'Descriptor',
       u'Location Type', u'Incident Zip', u'Incident Address', u'Street Name',
       u'Cross Street 1', u'Cross Street 2', u'Intersection Street 1',
       u'Intersection Street 2', u'Address Type', u'City', u'Landmark',
       u'Facility Type', u'Status', u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction'

In [17]:
CTtoNeigh = pd.read_csv('NTA and CT correspondence.csv')

In [18]:
CTtoNeigh.head()

,Unnamed: 0,BoroCT2010,Neighborhood
0,0,2000100,Rikers Island
1,2,2000200,Soundview-Castle Hill-Clason Point-Harding Park
2,5,2000400,Soundview-Castle Hill-Clason Point-Harding Park
3,10,2001600,Soundview-Castle Hill-Clason Point-Harding Park
4,14,2001900,Mott Haven-Port Morris


In [19]:
complete_2014m = pd.merge(complete_2014, CTtoNeigh, on='BoroCT2010')

In [20]:
len(complete_2014m)

1673507

In [21]:
grouped_bytype= pd.DataFrame(complete_2014m['Complaint Type'].groupby(complete_2014m['Neighborhood']).value_counts()).reset_index()
grouped_bytype.rename(columns={'level_1': 'complain_type',0: 'calls_bytype'}, inplace=True)
grouped2=grouped_bytype.groupby(grouped_bytype['Neighborhood']).sum().reset_index()
grouped2.rename(columns={0: 'total_calls'}, inplace=True)

In [22]:
grouped_bytype.head()

,Neighborhood,complain_type,calls_bytype
0,Airport,Taxi Complaint,670
1,Airport,For Hire Vehicle Complaint,152
2,Airport,Highway Condition,52
3,Airport,Street Condition,29
4,Airport,Taxi Report,28


In [23]:
grouped2.columns = ['Neighborhood', 'total_calls']

In [24]:
grouped2.head()

,Neighborhood,total_calls
0,Airport,1038
1,Allerton-Pelham Gardens,4727
2,Annadale-Huguenot-Prince's Bay-Eltingville,4878
3,Arden Heights,3766
4,Astoria,14200


In [25]:
df2=grouped_bytype.pivot(index='Neighborhood',columns='complain_type',values='calls_bytype')
df2.reset_index(inplace=True)
df2.head()
#df2.to_csv('2014_bytype.csv')
df3=pd.merge(df2,grouped2,on=['Neighborhood'],how='inner')
print len(df2), len(grouped2),len(df3)
df3.head()

195 195 195


complain_type,Neighborhood,AGENCY,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,...,Vacant Lot,Vending,Violation of Park Rules,WATER LEAK,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment,total_calls
0,Airport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,1038
1,Allerton-Pelham Gardens,NaN,13.0,NaN,10.0,57.0,NaN,NaN,2.0,1.0,...,7.0,2.0,NaN,60.0,31.0,4.0,221.0,NaN,NaN,4727
2,Annadale-Huguenot-Prince's Bay-Eltingville,NaN,4.0,NaN,8.0,30.0,2.0,1.0,3.0,1.0,...,25.0,1.0,2.0,5.0,29.0,15.0,272.0,NaN,NaN,4878
3,Arden Heights,NaN,3.0,NaN,3.0,22.0,NaN,1.0,2.0,NaN,...,1.0,NaN,NaN,4.0,49.0,11.0,373.0,NaN,NaN,3766
4,Astoria,NaN,51.0,1.0,45.0,105.0,NaN,1.0,14.0,4.0,...,9.0,35.0,1.0,132.0,31.0,22.0,338.0,1.0,NaN,14200


In [26]:
df3.columns

Index([u'Neighborhood', u'AGENCY', u'APPLIANCE', u'Adopt-A-Basket',
       u'Air Quality', u'Animal Abuse', u'Animal Facility - No Permit',
       u'Animal in a Park', u'Asbestos', u'BEST/Site Safety',
       ...
       u'Vacant Lot', u'Vending', u'Violation of Park Rules', u'WATER LEAK',
       u'Water Conservation', u'Water Quality', u'Water System',
       u'Window Guard', u'X-Ray Machine/Equipment', u'total_calls'],
      dtype='object', name=u'complain_type', length=181)

In [27]:
attributes_norm=dataset_2014_[dataset_2014_.columns[3:23]]
attributes_norm['Neighborhood']=dataset_2014_['Neighborhood']

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [28]:
callsbytype_attributes=pd.merge(attributes_norm,df3,on=['Neighborhood'],how='inner')
callsbytype_attributes.head()

,Median Age,White only,Black only,Median Rent,Median Income,Normalized cars,Normalized High school,Normalized bachelors,Normalized masters,Normalized PHD,...,Vacant Lot,Vending,Violation of Park Rules,WATER LEAK,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment,total_calls
0,39.70,11779.0,11280.0,1538.0,64702.0,1.129863,170.556606,103.757977,52.558497,7.356417,...,7.0,2.0,NaN,60.0,31.0,4.0,221.0,NaN,NaN,4727
1,41.60,27441.0,152.0,1342.0,91354.5,1.858472,210.446054,145.184210,57.879597,5.164108,...,25.0,1.0,2.0,5.0,29.0,15.0,272.0,NaN,NaN,4878
2,40.45,23119.0,410.0,1299.0,88864.5,1.741741,188.149363,152.136552,69.603937,5.507382,...,1.0,NaN,NaN,4.0,49.0,11.0,373.0,NaN,NaN,3766
3,34.10,53369.0,4421.0,1432.0,57798.0,0.422238,141.163500,217.835138,89.772359,10.999416,...,9.0,35.0,1.0,132.0,31.0,22.0,338.0,1.0,NaN,14200
4,42.00,10271.0,313.0,1711.0,71800.0,1.453384,169.150912,156.102672,89.432830,12.095814,...,1.0,3.0,NaN,2.0,22.0,5.0,328.0,NaN,NaN,3578


In [29]:
len(callsbytype_attributes)

190

In [30]:
regressors=attributes_norm.columns[0:20]
attributes_norm.columns

Index([u'Median Age', u'White only', u'Black only', u'Median Rent',
       u'Median Income', u'Normalized cars', u'Normalized High school',
       u'Normalized bachelors', u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households', u'Neighborhood'],
      dtype='object')

In [31]:
regressors

Index([u'Median Age', u'White only', u'Black only', u'Median Rent',
       u'Median Income', u'Normalized cars', u'Normalized High school',
       u'Normalized bachelors', u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households', u'Neighborhood'],
      dtype='object')

In [32]:
#attributes
types_of_calls=df3.columns[1:180]
df3.columns

Index([u'Neighborhood', u'AGENCY', u'APPLIANCE', u'Adopt-A-Basket',
       u'Air Quality', u'Animal Abuse', u'Animal Facility - No Permit',
       u'Animal in a Park', u'Asbestos', u'BEST/Site Safety',
       ...
       u'Vacant Lot', u'Vending', u'Violation of Park Rules', u'WATER LEAK',
       u'Water Conservation', u'Water Quality', u'Water System',
       u'Window Guard', u'X-Ray Machine/Equipment', u'total_calls'],
      dtype='object', name=u'complain_type', length=181)

##Lasso Regression for each type

###first, the code for just one type, then I will write the code for every type of complain

In [33]:
types_of_calls[20]

'Bus Stop Shelter Placement'

In [34]:
typeof=types_of_calls[20]

In [35]:
A2=np.append(np.append(np.append(regressors,typeof),'Neighborhood'),'total_calls')
myframe1=callsbytype_attributes[A2].dropna()

In [36]:
print len(myframe1)
myframe1.head()

100


,Median Age,White only,Black only,Median Rent,Median Income,Normalized cars,Normalized High school,Normalized bachelors,Normalized masters,Normalized PHD,Normalized familiy household,Normalized non-family household,Normalized owner ocuppied,Normalized renter ocuppied,Total Population,Total households,Neighborhood,Bus Stop Shelter Placement,Neighborhood,total_calls
3,34.1,53369.0,4421.0,1432.0,57798.0,0.422238,141.163500,217.835138,89.772359,10.999416,0.467531,0.532469,0.170327,0.829673,77095.0,34310.0,Astoria,2.0,Astoria,14200
6,38.9,16758.0,472.0,1241.0,55280.0,0.791371,187.863259,138.460096,56.933942,6.218361,0.710562,0.289438,0.456359,0.543641,32002.0,11125.0,Bath Beach,1.0,Bath Beach,5532
7,33.3,29923.0,1329.0,2000.0,134435.0,0.275123,26.574085,331.010945,180.040014,29.351536,0.387302,0.612698,0.214546,0.785454,42485.0,21216.0,Battery Park City-Lower Manhattan,1.0,Battery Park City-Lower Manhattan,8712
8,38.3,61867.0,2047.0,1329.5,60717.5,0.689663,147.585831,181.788036,88.762302,12.896884,0.570461,0.429539,0.376677,0.623323,84439.0,34991.0,Bay Ridge,3.0,Bay Ridge,16316
10,31.2,26391.0,35836.0,1226.0,38566.0,0.354896,134.340223,99.810337,52.812875,4.337154,0.574117,0.425883,0.202334,0.797666,71706.0,24920.0,Bedford,4.0,Bedford,15810


In [37]:
myframe1.columns

Index([u'Median Age', u'White only', u'Black only', u'Median Rent',
       u'Median Income', u'Normalized cars', u'Normalized High school',
       u'Normalized bachelors', u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households', u'Neighborhood',
       u'Bus Stop Shelter Placement', u'Neighborhood', u'total_calls'],
      dtype='object')

In [38]:
X=myframe1[['Median Age', u'White only', u'Black only', u'Median Rent',
       u'Median Income', u'Normalized cars', u'Normalized High school',
       u'Normalized bachelors', u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households']]
Y=myframe1[typeof]

In [39]:
X_scaled = preprocessing.scale(X)
Y_scaled = preprocessing.scale(Y)

In [40]:
#LASSO
X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.30, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
R2_store=[]
for i in range(-40,40):
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
    R2_store.append(R2_OS)
optim_alpha=range(-40,40)[np.where(R2_store==np.max(R2_store))[0][0]]
print "alpha = %f"%optim_alpha
Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
Lasso.fit(X_train,Y_train)
R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
print R2_OS

alpha = 1.000000
3.33066907388e-16


C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


###Now, for every type:

In [41]:
results={}
for typeof in types_of_calls:
    A2=np.append(np.append(np.append(regressors,typeof),'Neighborhood'),'total_calls')
    myframe1=callsbytype_attributes[A2].dropna()
    if len(myframe1)>100:
        results[typeof]={}
        X=myframe1[['Median Age', u'White only', u'Black only', u'Median Rent',
       u'Median Income', u'Normalized cars', u'Normalized High school',
       u'Normalized bachelors', u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households']]
        Y=myframe1[typeof]
        X_scaled = preprocessing.scale(X)
        Y_scaled = preprocessing.scale(Y)
        #LASSO
        X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.30, random_state=1)
        X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
        R2_store=[]
        for i in range(-40,40):
            Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
            Lasso.fit(X_train,Y_train)
            R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
            R2_store.append(R2_OS)
        optim_alpha=range(-40,40)[np.where(R2_store==np.max(R2_store))[0][0]]
        results[typeof]['best_alpha']=optim_alpha
        Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
        Lasso.fit(X_train,Y_train)
        R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
        results[typeof]['best_R2']=R2_OS
        results[typeof]['sample']=len(myframe1)

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


In [42]:
results_frame = pd.DataFrame(index=range(0,len(results.keys())), columns=['type','best_alpha','R2','sample_size'])

In [43]:
results_frame['type']=results.keys()
results_frame.head()
for i in results_frame.index:
    typeof=results_frame.loc[i,'type']
    results_frame.iloc[i,1]=results[typeof]['best_alpha']
    results_frame.iloc[i,2]=results[typeof]['best_R2']  
    results_frame.iloc[i,3]=results[typeof]['sample'] 

In [44]:
results_frame.sort('R2',ascending=False)

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,type,best_alpha,R2,sample_size
38,Noise,0,0.791357,190
76,Air Quality,0,0.671983,188
100,Sanitation Condition,0,0.668972,190
49,Sidewalk Condition,0,0.66177,189
40,Indoor Sewage,0,0.638016,178
82,ELECTRIC,0,0.592985,189
30,Taxi Complaint,0,0.57219,184
35,UNSANITARY CONDITION,0,0.545569,188
58,APPLIANCE,0,0.540976,187
2,Sweeping/Inadequate,0,0.540442,107


In [40]:
#results_frame.to_csv('Lasso_initial_results V3.csv')